## Install Neccesssary Libraries

In [1]:
pip install -r "../../requirements.txt"

Note: you may need to restart the kernel to use updated packages.


## Establish connection to API

In [1]:
from amadeus import Client, ResponseError
amadeus = Client(
    client_id='vsNE2cVrLzYVgWYcF1npWpv3HYA5ICs7',
    client_secret='rasieAFzfhNCXA0D'
)

## Connect to CSV file

In [2]:
import pandas as pd 
pd.set_option('display.max_row', 1000)
df = pd.read_csv("SafeIndex.csv") 

In [ ]:
neighbourhood_unique = df.neighbourhood_cleansed.unique()
#neighbourhood_unique

## Group and Sort listings by neighborhood

In [ ]:
# number of listings used to get the avg safe index score
k = 5

df_grouped = df.groupby('neighbourhood_cleansed')

# group the listings into neighborhoods
top_n_rows = df_grouped.head(k)

# sort the listings after grouping for better visulaization
top_n_rows_sorted = top_n_rows.sort_values(by=['neighbourhood_cleansed'], ignore_index=True)

# check if all records have more than k listing
records_per_neighborhood = top_n_rows_sorted.groupby(['neighbourhood_cleansed']).size().reset_index(name='counts')

# filter neighborhoods that have less than to predeinfe values, k
filtered_neighborhoods = records_per_neighborhood[records_per_neighborhood['counts'] >= k]

# get records that have more than k listing
final_records = top_n_rows_sorted[top_n_rows_sorted['neighbourhood_cleansed'].isin(filtered_neighborhoods.neighbourhood_cleansed.unique())]

## Get Response

In [ ]:
'''
# Flight Most Booked Destinations
amadeus.travel.analytics.air_traffic.booked.get(originCityCode='MAD', period='2017-08')

# Flight Most Traveled Destinations
amadeus.travel.analytics.air_traffic.traveled.get(originCityCode='MAD', period='2017-01')

# Flight Busiest Travel Period
amadeus.travel.analytics.air_traffic.busiest_period.get(cityCode='MAD', period='2017', direction='ARRIVING')

# Safe Place
# How safe is Barcelona? (based a geo location and a radius)
amadeus.safety.safety_rated_locations.get(latitude=41.397158, longitude=2.160873)
# How safe is Barcelona? (based on a square)
amadeus.safety.safety_rated_locations.by_square.get(north=41.397158, west=2.160873,
                                                    south=41.394582, east=2.177181)
# What is the safety information of a location based on it's Id?
amadeus.safety.safety_rated_location('Q930400801').get()
'''

In [ ]:
safeCategory = ['overall', 'lgbtq', 'medical', 'physicalHarm', 'politicalFreedom', 'theft', 'women']

# for each neighborhood creating empty dict with all the safe category
safeIndexScoring = { j: {k:0 for k in safeCategory} for j in filtered_neighborhoods.neighbourhood_cleansed.unique()}

### Api calling

In [ ]:
# for index, row in final_records.iterrows():
#     # retrieve data from Api
#     response = amadeus.safety.safety_rated_locations.get(latitude=row['latitude'], longitude=row['longitude'])
    
#     # add the each safe category score to the respective category in dictionary 
#     for cat in safeCategory:
#         safeIndexScoring[row['neighbourhood_cleansed']][cat] += response.data[0]['safetyScores'][cat]
        

In [ ]:
# now that we have the total safety score we need to avg them to get the avg safety score
safeIndexScoring_copy= safeIndexScoring.copy()

denominator = filtered_neighborhoods.values.tolist()
tmp = {}

for area,listingCount in denominator:
    tmp[area] = {cat: round(total/listingCount,2) for cat, total in safeIndexScoring_copy[area].items()}

### Store result into Pickle file

In [ ]:
output = pd.concat({
        k: pd.DataFrame.from_dict(v, 'index') for k, v in tmp.items()
    }, 
    axis=0)
output.to_pickle("./safeIndex.pkl")

### Read Pickle File

In [3]:
unpickled_df = pd.read_pickle("safeIndex.pkl")
unpickled_df

0
Bayview               overall           51.8
                      lgbtq             46.0
                      medical           76.6
                      physicalHarm      46.4
                      politicalFreedom  31.4
                      theft             68.8
                      women             38.6
Bernal Heights        overall           51.0
                      lgbtq             46.0
                      medical           75.0
                      physicalHarm      46.0
                      politicalFreedom  31.0
                      theft             68.0
                      women             37.0
Castro/Upper Market   overall           37.8
                      lgbtq             30.0
                      medical           71.2
                      physicalHarm      28.8
                      politicalFreedom  21.8
                      theft             51.8
                      women             23.8
Chinatown             overall           35.2
                      lgbtq             27.0
                      medical           70.0
                      physicalHarm      25.2
                      politicalFreedom  20.6
                      theft             48.6
                      women             21.2
Crocker Amazon        overall           48.0
                      lgbtq             43.0
                      medical           73.0
                      physicalHarm      43.0
                      politicalFreedom  28.0
                      theft             66.0
                      women             32.0
Diamond Heights       overall           51.0
                      lgbtq             46.0
                      medical           75.0
                      physicalHarm      46.0
                      politicalFreedom  31.0
                      theft             68.0
                      women             37.0
Downtown/Civic Center overall           40.0
                      lgbtq             33.0
                      medical           72.0
                      physicalHarm      32.0
                      politicalFreedom  23.0
                      theft             55.0
                      women             26.0
Excelsior             overall           51.0
                      lgbtq             46.0
                      medical           75.0
                      physicalHarm      46.0
                      politicalFreedom  31.0
                      theft             68.0
                      women             37.0
Financial District    overall           37.6
                      lgbtq             30.0
                      medical           71.0
                      physicalHarm      28.6
                      politicalFreedom  21.8
                      theft             51.8
                      women             23.6
Glen Park             overall           51.0
                      lgbtq             46.0
                      medical           75.0
                      physicalHarm      46.0
                      politicalFreedom  31.0
                      theft             68.0
                      women             37.0
Haight Ashbury        overall           35.6
                      lgbtq             27.0
                      medical           70.4
                      physicalHarm      25.6
                      politicalFreedom  20.6
                      theft             48.6
                      women             21.6
Inner Richmond        overall           42.0
                      lgbtq             33.8
                      medical           72.2
                      physicalHarm      34.8
                      politicalFreedom  24.2
                      theft             57.6
                      women             29.4
Inner Sunset          overall           28.6
                      lgbtq             16.8
                      medical           67.2
                      physicalHarm      16.4
                      politicalFreedom